# Host-guest usage with stk

by: Andrew Tarzia

# Imports 

In [1]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

from rdkit import Chem 
from rdkit.Chem import AllChem as rdkit
from collections import defaultdict
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDistGeom
IPythonConsole.ipython_3d = True

import py3Dmol
from IPython.display import Image
import matplotlib.pyplot as plt
import subprocess
import time
import stk
import stko
import spindry as spd
%matplotlib inline

# Some useful functions 

In [2]:
def show_stk_mol(stk_mol):
    data = rdkit.MolToMolBlock(stk_mol.to_rdkit_mol())
    p = py3Dmol.view(
        data=data,
        style={'stick':{'colorscheme':'cyanCarbon'}}, 
        width=400,
        height=400,
    )
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    p.show()

Working:

In [8]:
# Produce a Pd+2 atom with 4 functional groups.
palladium_atom = stk.BuildingBlock(
    smiles='[Pd+2]',
    functional_groups=(
        stk.SingleAtom(stk.Pd(0, charge=2))
        for i in range(4)
    ),
    position_matrix=[[0., 0., 0.]],
)

# Build a building block with two functional groups using
# the SmartsFunctionalGroupFactory.
bb1 = stk.BuildingBlock(
    smiles=(
        'C1=NC=CC(C2=CC=CC(C3=CC=NC=C3)=C2)=C1'
    ),
    functional_groups=[
        stk.SmartsFunctionalGroupFactory(
            smarts='[#6]~[#7X2]~[#6]',
            bonders=(1, ),
            deleters=(),
        ),
    ],
)

cage1 = stk.ConstructedMolecule(
    stk.cage.M6L12Cube(
        building_blocks=(palladium_atom, bb1),
        # Ensure that bonds between the GenericFunctionalGroups
        # of the ligand and the SingleAtom functional groups
        # of the metal are dative.
        reaction_factory=stk.DativeReactionFactory(
            stk.GenericReactionFactory(
                bond_orders={
                    frozenset({
                        stk.GenericFunctionalGroup,
                        stk.SingleAtom
                    }): 9
                }
            )
        ),
        optimizer=stk.MCHammer(num_steps=2000),
    )
)

In [9]:
show_stk_mol(cage1)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [10]:
stk_guest = stk.BuildingBlock('C1C(O[H])C2C(C=CC(=C2)CP([H])[H])CC1')
show_stk_mol(stk_guest)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [11]:
cage_atoms = [
    (atom.get_id(), atom.__class__.__name__)
    for atom in cage1.get_atoms()
]
stk_mol_atoms = [
    (atom.get_id(), atom.__class__.__name__)
    for atom in stk_guest.get_atoms()
]

In [12]:
host = spd.Molecule.init(
    atoms=(
        spd.Atom(id=i[0], element_string=i[1])
        for i in cage_atoms
    ),
    position_matrix=cage1.get_position_matrix(),
)
guest = spd.Molecule.init(
    atoms=(
        spd.Atom(id=i[0], element_string=i[1])
        for i in stk_mol_atoms
    ),
    position_matrix=stk_guest.get_position_matrix(),
)

In [13]:
print(host, guest)

<Molecule(366 atoms) at 140087873842480> <Molecule(30 atoms) at 140087827908424>


In [14]:
cg = spd.Spinner(
    step_size=0.5,
    rotation_step_size=5,
    num_conformers=100,
    max_attempts=2000,
)

In [18]:
energies = {}
for conformer in cg.get_conformers(host, guest):
    print(conformer)
    print(conformer.get_cid(), conformer.get_potential())
    cage1 = cage1.with_position_matrix(
        conformer.get_host().get_position_matrix()
    )
    stk_guest = stk_guest.with_position_matrix(
        conformer.get_guest().get_position_matrix()
    )
    complex_mol = stk.ConstructedMolecule(
        topology_graph=stk.host_guest.Complex(stk.BuildingBlock.init_from_molecule(cage1), stk_guest)
    )
    complex_mol.write(
        f'hg_example_output/hg_conf_{conformer.get_cid()}.mol'
    )
    energies[conformer.get_cid()] = stko.UFFEnergy(ignore_inter_interactions=False).get_energy(complex_mol)

<SupraMolecule(366 + 30 atoms) at 140087839905272>
0 551.71811150978
<SupraMolecule(366 + 30 atoms) at 140087873803432>
1 551.71811150978
<SupraMolecule(366 + 30 atoms) at 140087839927152>
2 543.4115335823742


RDKit ERROR: [12:02:49] UFFTYPER: Unrecognized atom type: Pd6 (0)
RDKit ERROR: [12:02:49] UFFTYPER: Unrecognized atom type: Pd6 (1)
RDKit ERROR: [12:02:49] UFFTYPER: Unrecognized atom type: Pd6 (2)
RDKit ERROR: [12:02:49] UFFTYPER: Unrecognized atom type: Pd6 (3)
RDKit ERROR: [12:02:49] UFFTYPER: Unrecognized atom type: Pd6 (4)
RDKit ERROR: [12:02:49] UFFTYPER: Unrecognized atom type: Pd6 (5)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (0)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (1)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (2)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (3)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (4)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (5)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (0)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (1)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (2)
RDKit ERRO

<SupraMolecule(366 + 30 atoms) at 140087827931264>
3 537.8218710173678
<SupraMolecule(366 + 30 atoms) at 140087839904992>
4 538.9594953327573


RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (0)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (1)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (2)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (3)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (4)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (5)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (0)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (1)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (2)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (3)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (4)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (5)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (0)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (1)
RDKit ERROR: [12:02:50] UFFTYPER: Unrecognized atom type: Pd6 (2)
RDKit ERRO

<SupraMolecule(366 + 30 atoms) at 140087827931264>
5 534.3715550685081
<SupraMolecule(366 + 30 atoms) at 140087873803432>
6 532.3879145182494


...

<SupraMolecule(366 + 30 atoms) at 140087827070760>
33 529.9549114678946
<SupraMolecule(366 + 30 atoms) at 140087828257984>
34 530.5035304450796


RDKit ERROR: [12:02:54] UFFTYPER: Unrecognized atom type: Pd6 (0)
RDKit ERROR: [12:02:54] UFFTYPER: Unrecognized atom type: Pd6 (1)
RDKit ERROR: [12:02:54] UFFTYPER: Unrecognized atom type: Pd6 (2)
RDKit ERROR: [12:02:54] UFFTYPER: Unrecognized atom type: Pd6 (3)
RDKit ERROR: [12:02:54] UFFTYPER: Unrecognized atom type: Pd6 (4)
RDKit ERROR: [12:02:54] UFFTYPER: Unrecognized atom type: Pd6 (5)
RDKit ERROR: [12:02:55] UFFTYPER: Unrecognized atom type: Pd6 (0)
RDKit ERROR: [12:02:55] UFFTYPER: Unrecognized atom type: Pd6 (1)
RDKit ERROR: [12:02:55] UFFTYPER: Unrecognized atom type: Pd6 (2)
RDKit ERROR: [12:02:55] UFFTYPER: Unrecognized atom type: Pd6 (3)
RDKit ERROR: [12:02:55] UFFTYPER: Unrecognized atom type: Pd6 (4)
RDKit ERROR: [12:02:55] UFFTYPER: Unrecognized atom type: Pd6 (5)


<SupraMolecule(366 + 30 atoms) at 140087839853984>
35 529.1909769887152


RDKit ERROR: [12:02:55] UFFTYPER: Unrecognized atom type: Pd6 (0)
RDKit ERROR: [12:02:55] UFFTYPER: Unrecognized atom type: Pd6 (1)
RDKit ERROR: [12:02:55] UFFTYPER: Unrecognized atom type: Pd6 (2)
RDKit ERROR: [12:02:55] UFFTYPER: Unrecognized atom type: Pd6 (3)
RDKit ERROR: [12:02:55] UFFTYPER: Unrecognized atom type: Pd6 (4)
RDKit ERROR: [12:02:55] UFFTYPER: Unrecognized atom type: Pd6 (5)
RDKit ERROR: [12:02:55] UFFTYPER: Unrecognized atom type: Pd6 (0)
RDKit ERROR: [12:02:55] UFFTYPER: Unrecognized atom type: Pd6 (1)
RDKit ERROR: [12:02:55] UFFTYPER: Unrecognized atom type: Pd6 (2)
RDKit ERROR: [12:02:55] UFFTYPER: Unrecognized atom type: Pd6 (3)
RDKit ERROR: [12:02:55] UFFTYPER: Unrecognized atom type: Pd6 (4)
RDKit ERROR: [12:02:55] UFFTYPER: Unrecognized atom type: Pd6 (5)


<SupraMolecule(366 + 30 atoms) at 140087827951912>
36 528.9362135171814
37 conformers generated in 1999 steps.


In [19]:
energies

{0: 7466.813616031522,
 1: 7466.813616031522,
 2: 7466.571841307307,
 3: 7466.933053453889,
 4: 7467.060240212348,
 5: 7467.386981120916,
 6: 7467.650394312734,
 7: 7467.607040526277,
 8: 7467.701461060949,
 9: 7467.534417978909,
 10: 7467.446644690921,
 11: 7467.614490528407,
 12: 7467.553509769047,
 13: 7467.664291429686,
 14: 7467.695157451185,
 15: 7467.7085786829175,
 16: 7467.719387312567,
 17: 7467.653613717966,
 18: 7467.6777631368395,
 19: 7467.623359581566,
 20: 7467.623686020108,
 21: 7467.718044948435,
 22: 7467.458980677772,
 23: 7467.409199906842,
 24: 7467.584086379514,
 25: 7467.717984340709,
 26: 7467.77552263083,
 27: 7467.729907181044,
 28: 7467.800852725664,
 29: 7467.785530747207,
 30: 7467.8117105117735,
 31: 7467.732724257994,
 32: 7467.695867643638,
 33: 7467.815780290201,
 34: 7467.531527384651,
 35: 7467.807198762412,
 36: 7467.750309416206}